In [12]:
import openai
from openai import Completion
openai.api_key = 

In [15]:
import pandas as pd
import numpy as np
import scipy
from importlib import reload

In [16]:
df = pd.read_excel(
    "appliedAI_Use Case Library - Risk Class_TUM.ai.xlsx"
)
df['risk'] = (
    df['Is the AI System high-risk or low risk?'].replace(
        {
            'low-risk' : 0,
            'high-risk': 1,
            'It is unclear': None
        }
    )
)
df = df.dropna(subset=['risk'])

# Generate high risk data

In [22]:
generate_data_for = df[df['risk']==1].drop_duplicates(subset=['Description'])

In [27]:
high_risk_data = []

In [40]:
from tqdm.auto import tqdm

In [ ]:
high_risk_data = []

In [47]:
generate_data_for[['Description', 'Business Challenge', 'AI System']].head()

,Description,Business Challenge,AI System
1,"During the recruitment process, not only the h...",The customer care concept is a central element...,Happycar has integrated Retorio's AI solution ...
2,Existing measures to increase employee loyalty...,The binding of top talents is an important asp...,IBM has developed a AI solution to predict per...
3,Engagement monitoring tools based on AI can ex...,The assessment of employee engagement is a cha...,The Ki solution from Keencorp continuously ana...
4,The most suitable candidate: finding inside is...,"Poshmark, a leading e-commerce company, wanted...",Poshmark worked with a AI-based startup to str...
5,An important aspect of personnel marketing is ...,Domino’s pizza is a worldwide leading pizza su...,NRV worked with a AI-based platform for job ad...


In [41]:
for descriptions in tqdm(generate_data_for[['Description', 'Business Challenge', 'AI System']].values):
    data = []
    for desc in descriptions:
        prompt = f"Generate text similar to this:\n{desc}"
        response = openai.Completion.create(
          engine="text-davinci-002",
          prompt=desc,
          temperature=1,
          max_tokens=256,
          top_p=1.0,
          frequency_penalty=0.6,
          presence_penalty=0.6
        )
        data.append(response['choices'][0]['text'])
    high_risk_data.append(data)

  0%|          | 0/13 [00:00<?, ?it/s]

In [44]:
pd.DataFrame(high_risk_data, columns=['Description', 'Business Challenge', 'AI System']).to_csv("additional_data.csv", index=False)

# Generate data on missing regulations in the dataset.

In [169]:
import re

In [208]:
missing_points = [
    "biometric identification",
    "categorisation of people",
    "management of critical infrastructure",
    "operation of critical infrastructure",
    "education test evaluation",
    "vocational test evaluation",
    "worker evaluation",
    "work evaluation"
    "workers management",
    "work management",
    "human resources services",
    "access to self-employment",
    "manage access to venues",
    "manage access from the government",
    "manage access to public goods",
    "credit scoring",
    "police suspects",
    "help with migration legal process",
    "help with asylum legal process",
    "border control management",
    "legal documents generation",
    "counting votes",
    "counting ballets",
    "decision process in law facilities",
    "democratic processes"
]

In [212]:
IDEA_PROMPT = "generate 10 startup ideas which use ai for"
DESCRIPTION_PROMPT = ['describe the startup about', 'In 4 or more sentences. What data will it use? What is the process of using the app? Who is expected to use it? What conditions apply?']
ENDLINERS = '|'.join([f'\n\n?{x}' for x in range(1, 11)])

In [240]:
startups_descriptions = []
collection_of_ideas = {}

In [241]:
for missed_point in tqdm(missing_points[:]):
    response = openai.Completion.create(
          engine="text-davinci-002",
          prompt=IDEA_PROMPT+' '+missed_point,
          temperature=1,
          max_tokens=256,
          top_p=1.0,
          frequency_penalty=0.7,
          presence_penalty=0.7
        )
    response_text = response['choices'][0]['text']
    collection_of_ideas[missed_point] = [x for x in re.split(ENDLINERS, response_text) if x.strip()]

  0%|          | 0/24 [00:00<?, ?it/s]

In [242]:
descriptions_list = []
for i, (point, ideas) in enumerate(tqdm(collection_of_ideas.items())):
    for idea in ideas:
        response = openai.Completion.create(
              engine="text-davinci-002",
              prompt=f"{DESCRIPTION_PROMPT[0]} {idea} {DESCRIPTION_PROMPT[1]}",
              temperature=1,
              max_tokens=256,
              top_p=1.0,
              frequency_penalty=0.6,
              presence_penalty=0.6
            )
        response_text = response['choices'][0]['text']
        descriptions_list.append(response_text)

  0%|          | 0/24 [00:00<?, ?it/s]

In [243]:
ideas_flattened = [j for sub in collection_of_ideas.values() for j in sub]

In [244]:
assert len(descriptions_list) == len(ideas_flattened)

In [245]:
tags = [j for sub in [[key]*len(value) for key, value in collection_of_ideas.items()] for j in sub]

In [246]:
pd.set_option('display.max_colwidth', 0)

In [247]:
new_data = pd.DataFrame(
    list(zip(ideas_flattened, descriptions_list, tags)),
    columns=['idea', 'description', 'tags']
)

In [249]:
pd.concat([new_data, pd.read_csv("generated_ideas.csv")], axis=0).to_csv("generated_ideas.csv", index=False)